# QField Integration into Machine Learning Landcover Classification

Welcome to this interactive notebook! We'll explore how to integrate QField-collected data into a machine learning workflow for landcover classification using **Digital Earth Pacific**.

## Notebook Structure

This notebook is divided into **three main parts**:

<details>
<summary><strong>1. Data Preparation</strong></summary>

- Define the Area Of Interest (AOI)
- Load data from the Digital Earth Pacific Catalog
- Interpolate on data points
</details>

<details>
<summary><strong>2. Model Training & Validation</strong></summary>

- Build a Random Forest classifier
- Split data into training, validation, and testing sets
- Train and fine-tune the model
</details>

<details>
<summary><strong>3. Visualize the Prediction</strong></summary>

- Display predictions from the trained model
- Compare with validation data
</details>


## Your Task

Follow the notebook and **fill in the missing code** where indicated. Look out for cells marked with `# ⚠️ TODO` or `# Insert your code here`.  
Answer the questions Look out for cells marked with `❓ Question for you: `

We'll guide you step-by-step. Let's get started!

## Importing Required Libraries

In [1]:
# Geospatial and data handling
import geopandas as gpd  # For vector data
import numpy as np       # Numerical operations
import pandas as pd      # Tabular data
import xarray as xr      # Multi-dimensional arrays

# Distributed computing
from dask.distributed import Client as DaskClient

# Digital Earth Pacific STAC access
from odc.stac import configure_s3_access, load
from pystac_client import Client
import requests
from odc.geo.geom import BoundingBox

# Machine Learning
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, cohen_kappa_score,
    classification_report, confusion_matrix
)

# Visualization
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# Custom utility functions
from utils import load_data, mask_cloud, add_indices

print("All libraries imported successfully!")

All libraries imported successfully!


In [2]:
# Configure S3 access for Digital Earth Pacific STAC
configure_s3_access(aws_unsigned=True)
print("S3 access configured (unsigned)")

# Set up a Dask client for parallel processing
# You can adjust the number of workers and memory based on your system
dask_client = DaskClient(
    n_workers=2,
    threads_per_worker=16,
    memory_limit="30GB",
)

print(f"Dask client started with {dask_client.nthreads()} threads across {len(dask_client.scheduler_info()['workers'])} workers")

S3 access configured (unsigned)
Dask client started with {'tcp://127.0.0.1:35321': 16, 'tcp://127.0.0.1:39679': 16} threads across 2 workers


## Import Data Catalog

We will be using the **Element 84 Earth Search Catalog**, an open **STAC (SpatioTemporal Asset Catalog)** API hosted on **AWS (Amazon Web Services)**.

This catalog provides access to a wide range of public Earth observation datasets, including:

- **Sentinel-2**
- **Landsat**
- Other public satellite imagery

[Explore the Element 84 STAC Catalog](https://stacindex.org/catalogs/earth-search#/)

STAC is a standardized way to describe geospatial assets. It allows us to:

- Search for satellite imagery by time, location, and properties
- Load data efficiently into our analysis workflows
- Interoperate with cloud-native geospatial tools

> **Question for you:**  
> What types of landcover changes are you most interested in detecting using this data?
>
> ##### Record your responses here: 

In [3]:
# Connect to the Element 84 STAC Catalog
catalog_url = "https://earth-search.aws.element84.com/v1/"
client = Client.open(catalog_url)
print("Connected to STAC catalog")

# List available collections
response = requests.get(f"{catalog_url}collections")
collections = response.json()

print(f"Found {len(collections['collections'])} collections:\n")

# Display collection IDs
for c in collections["collections"]:
    print(f"• {c['id']}")

Connected to STAC catalog
Found 9 collections:

• sentinel-2-pre-c1-l2a
• cop-dem-glo-30
• naip
• cop-dem-glo-90
• landsat-c2-l2
• sentinel-2-l2a
• sentinel-2-l1c
• sentinel-2-c1-l2a
• sentinel-1-grd


## 1. Data Preparation
In this section, we will:

1.1. **Define the Area of Interest (AOI)**  
1.2. **Load and process the following datasets**:
   - **Sentinel-2** imagery
   - **Spectral Indexes**: NDVI, NDBI
   - **Sentinel-1** radar data
   - **Elevation Model**  

1.3. **Merge all datasets** into a single multiband dataset.  
1.4. **Interpolate** the dataset over labeled data points.



### 1.1. Define Area of Interest 

##### ⚠️ **TODO** : 
Play with latitude and longitude values to choose your AOI (over Auckland). The example below return an AOI over whole New-Zealand, which is too big.

In [5]:
# Define Area of Interest (AOI)
# This bounding box covers a small region near Auckland, New Zealand
from odc.geo.geom import BoundingBox

# ⚠️ TODO : modify value left, bottom, right, top
bbox = BoundingBox(
    left=165.67,   # Longitude (West)
    bottom=-46.94, # Latitude (South)
    right=179.22,  # Longitude (East)
    top=-33.08     # Latitude (North)
)

# Explore the AOI interactively
bbox.explore()

### 1.2 Load and Process datasets : 

### Sentinel-2 Data

**Sentinel-2** is a land monitoring mission composed of two satellites that provide high-resolution optical imagery. Key features include:

- **Global coverage** every 5 days (or 8 days in the Pacific)
- **L2A data** available globally since **January 2017**
- **13 spectral bands**:
  - 4 visible (RGB)
  - 6 Near-Infrared (NIR)
  - 3 Short-Wave Infrared (SWIR)

#### What We'll Use

For this workshop, we will:

- Load **Sentinel-2 data from 2024**
- Focus on **RGB**, **NIR**, and **SWIR** bands
- Compute the **median composite** to:
  - Simplify the dataset (no temporal analysis)
  - Reduce cloud contamination (a major issue in tropical Pacific regions)

**Why median?**  
Cloud cover is a persistent challenge in optical satellite imagery. By taking the median across multiple observations, we reduce the impact of clouds and improve data quality for classification.

> ❓ **Question for you:**  
> Have you worked with Sentinel-2 composites before? What challenges did you face with cloud masking?
>
> > ##### Record your responses here: 

In [ ]:
# Load Sentinel-2 L2A data for the year 2024 over the defined AOI
spectral = load_data("sentinel-2-l2a", '2024', bbox)
print("Sentinel-2 data loaded")

# Apply cloud masking to clean the dataset
spectral = mask_cloud(spectral, "sentinel-2-l2a")
print("Cloud masking applied")

# Inspect the dataset
print(spectral)

In [ ]:
# Visualize the Red band from Sentinel-2
# This helps us inspect the spatial coverage and quality of the data
spectral.red.odc.explore()

##### ⚠️ **TODO**:

Explore other spectral bands by using the same command as above but changing the bands to:
   - green
   - near infrared

In [ ]:
# ⚠️ TODO : Explore other spectral




> ❓ **Question for you:**  
> Do you seen anything specific patterns ?
> > ##### Record your responses here: 

### Vegetation and Built-up Index

Spectral indices are powerful tools for highlighting specific landcover patterns. There are many types — for water, vegetation, wildfire, urban areas, and more.

In this workshop, we focus on **two indices** that are particularly useful for analyzing patterns in the Auckland region:

### Normalized Difference Vegetation Index (**NDVI**)

NDVI helps distinguish **vegetated** from **non-vegetated** areas.

**Formula:**

$$NDVI = \frac{NIR - RED}{NIR + RED}$$

- **High NDVI** (close to +1): Dense, healthy vegetation  
- **Low or negative NDVI**: Bare soil, built-up areas, or water

### Normalized Difference Built-up Index (**NDBI**)

NDBI highlights **built-up or impervious surfaces** like buildings and roads.

**Formula:**

$$NDBI = \frac{SWIR - NIR}{SWIR + NIR}$$

- **High NDBI**: Urban or built-up areas  
- **Low NDBI**: Vegetation or water bodies

> **Tip:**  
> These indices are derived from Sentinel-2 bands and will be used later to improve landcover classification accuracy.


In [ ]:
# Load and mask Sentinel-2 data
spectral = load_data("sentinel-2-l2a", '2024', bbox)
spectral = mask_cloud(spectral, "sentinel-2-l2a")

# Compute NDVI and NDBI indices
spectral_with_indices = add_indices(spectral).compute()

# Display the resulting dataset with indices
spectral_with_indices

In [ ]:
# Visualize NDVI using an intuitive colormap
# The 'RdYlGn' colormap helps distinguish vegetation density:
# - Green: High NDVI (dense vegetation)
# - Yellow: Moderate NDVI
# - Red: Low NDVI (bare soil, urban areas, or water)

spectral_with_indices.ndvi.odc.explore(cmap="RdYlGn")

### Your Turn: Explore Built-up Areas

Now that you've seen how NDVI highlights vegetation, let's explore **NDBI**, which helps identify **urban and impervious surfaces**.

⚠️ **TODO**  
1. Ammend the command `spectral_with_indices.ndvi.odc.explore(cmap="RdYlGn")` to visualize NDBI.  
2. Try changing the `cmap` to `"Greys"`, `"OrRd"`, or `"inferno"`

> ❓ **Question for you:**  
> Which one gives the clearest contrast?

> ##### Record your responses here: 



Feel free to zoom in and inspect areas that look urban vs vegetated!

In [ ]:
# ⚠️ TODO : Visualize NDBI




### Sentinel-1: Radar for All Conditions

**Sentinel-1** is a dual-satellite mission equipped with **Synthetic Aperture Radar (SAR)**, which operates independently of sunlight and weather conditions, making it ideal for consistent Earth observation.

### Key Features:
- Operates **day and night**, in **all weather**
- Uses **C-band SAR** to measure surface backscatter
- Offers **dual polarization**: `VV` (vertical transmit & receive) and `VH` (vertical transmit, horizontal receive)
- Sensitive to:
  - Surface roughness
  - Moisture content
  - Viewing geometry

> ❓ **Questions for you:**  
> How might radar backscatter differ between:
> - Forest vs grassland?
> - Urban vs water?
> - Wet vs dry soil?

> ##### Record your responses here: 

We'll explore these differences using Sentinel-1 data next. Ready to load it?

In [ ]:
# Load Sentinel-1 GRD data for 2024 over the AOI
# Sentinel-1 provides radar backscatter data (VV and VH polarizations)

radar = load_data("sentinel-1-grd", '2024', bbox)
print("Sentinel-1 data loaded")

# Apply cloud masking (for consistency, though radar isn't affected by clouds)
radar = mask_cloud(radar, "sentinel-1-grd")
print("Cloud masking applied")

# Inspect the radar dataset
print(radar)

In [ ]:
# Visualize Sentinel-1 VV polarization
# VV (vertical transmit & receive) is useful for detecting surface texture and moisture.
# Try zooming in to see how urban vs vegetated areas differ in radar backscatter.

radar.vv.odc.explore()

### Elevation Model

The **Elevation Model** available through the Element 84 catalog provides global terrain data derived from sources like:

- **Copernicus DEM**
- **NASA’s SRTM**

These datasets represent the Earth's surface height in **meters**, and are valuable for:

- Topographic analysis
- Environmental modeling
- Enhancing landcover classification (e.g., distinguishing valleys from ridges)


**Why use elevation?**  
Elevation can influence vegetation types, urban development, and water flow, making it a useful feature in landcover classification.

We'll now load elevation data for our AOI and integrate it into our dataset.

In [ ]:
# Load Elevation Data (Copernicus DEM 30m)
# Elevation data is static, so no need to specify a year

dem_data = load_data("cop-dem-glo-30", None, bbox)

# Inspect the elevation dataset
dem_data

In [ ]:
# Visualize Elevation Data
# This shows terrain elevation in meters across the AOI.
# Try zooming in to identify valleys, ridges, and flat areas.

dem_data.data.odc.explore()

### Your Turn: Explore Terrain

Now that you've loaded and visualised the elevation data, let's explore how terrain varies across the AOI.

> ❓ **Question for you:**
> - Where are the **highest elevations**?
> - Which areas are **flat or low-lying**?
> - How might elevation influence landcover types?
> - Can you spot any terrain features that might affect vegetation or urban development?

> ##### Record your responses here: 

### 1.3 Merge all datatasets 

Now that we've loaded all relevant datasets, including:

- Sentinel-2 bands
- Spectral indices (NDVI, NDBI)
- Sentinel-1 radar (VV & VH)
- Elevation model

We will **merge** them into a single multiband dataset.

In [ ]:
# Merge all datasets into one multiband dataset

# Remove the 'time' dimension to simplify merging
spectral_with_indices = spectral_with_indices.squeeze("time", drop=True)
dem_data = dem_data.squeeze("time", drop=True)
radar = radar.squeeze("time", drop=True)

# Combine spectral, radar, and elevation data
final_data = xr.merge([spectral_with_indices, radar, dem_data])

# Rename elevation band for clarity
final_data = final_data.rename({
    "data": "dem"  # Rename 'data' to 'dem' for elevation
})

# Inspect the final merged dataset
final_data

### 1.4 Interpolation of Data on Label Points 

### Label Data 

> ❓ **Questions for you:**  
> 1. Looking at the different previous datasets and bands, which specific patterns or features can you identify in each band or index? For example, which band highlights vegetation, water, urban areas, or other landscape features?
> 2. Based on the the previous question, what potential land cover classes could be relevant for a future land cover classification over Auckland? Propose four classes.

> ##### Record your responses here: 


Check if you have the right classes by running the following cell and explore the label points. 

In [ ]:
# Load label points from GeoPackage
# Make sure the file is in the correct location or update the path accordingly
gdf = gpd.read_file("lulc_auckland_v3.gpkg").to_crs(final_data.odc.crs)

# Print all Land Cover label to check if you were right
print(gdf['Land_Cover'].unique())

# Explore the label points interactively
gdf.explore()

The label data points were derived from the Land Cover Database v5.0 available via the LRIS Portal:

[LCDB v5.0 - Mainland New Zealand](https://lris.scinfo.org.nz/layer/104400-lcdb-v50-land-cover-database-version-50-mainhese were downloaded as a shapefile and processed using QGIS to fit our needs.

### Why Interpolate?

To prepare for model training, we need to assign values from each band to our **label points** (landcover polygons).  
We do this using **nearest-neighbor interpolation**, which assigns each label point the value of the **closest pixel** in each band.

This step ensures that every labeled point has a complete feature set for classification.

> **Tip:**  
> Nearest interpolation is fast and simple, but other methods (e.g., bilinear, cubic) may be used for smoother results in other contexts.

In [ ]:
# Interpolate multiband data onto label points using nearest-neighbor method
sampled = final_data.interp(
    x=("points", gdf.geometry.x.values),
    y=("points", gdf.geometry.y.values),
    method="nearest"
)

# Clean and format the interpolated dataset
df = sampled.to_dataframe().reset_index(drop=True)

# Add label information from the original GeoDataFrame
df["Class_id"] = gdf["Class_id"].values 
df["Land_Cover"] = gdf["Land_Cover"].values

# Remove unnecessary coordinate reference column
df = df.drop(["spatial_ref"], axis=1)

# Fill missing values with class-wise mean
df = df.groupby("Class_id", group_keys=False).apply(lambda g: g.fillna(g.mean(numeric_only=True)))

# Summary
print(f'Number of labeled points: {len(df)}')
df.head()

In [ ]:
# Merge all datasets into one multiband dataset

# Remove the 'time' dimension to simplify merging
spectral_with_indices = spectral_with_indices.squeeze("time", drop=True)
dem_data = dem_data.squeeze("time", drop=True)
radar = radar.squeeze("time", drop=True)

# Combine spectral, radar, and elevation data
final_data = xr.merge([spectral_with_indices, radar, dem_data])

# Rename elevation band for clarity
final_data = final_data.rename({
    "data": "dem"  # Rename 'data' to 'dem' for elevation
})

# Inspect the final merged dataset
final_data

In [ ]:
# Load label points from GeoPackage
# Make sure the file is in the correct location or update the path accordingly
gdf = gpd.read_file("lulc_auckland_v3.gpkg").to_crs(final_data.odc.crs)

# Explore the label points interactively
gdf.explore()

In [ ]:
# Interpolate multiband data onto label points using nearest-neighbor method
sampled = final_data.interp(
    x=("points", gdf.geometry.x.values),
    y=("points", gdf.geometry.y.values),
    method="nearest"
)

# Clean and format the interpolated dataset
df = sampled.to_dataframe().reset_index(drop=True)

# Add label information from the original GeoDataFrame
df["Class_id"] = gdf["Class_id"].values 
df["Land_Cover"] = gdf["Land_Cover"].values

# Remove unnecessary coordinate reference column
df = df.drop(["spatial_ref"], axis=1)

# Fill missing values with class-wise mean
df = df.groupby("Class_id", group_keys=False).apply(lambda g: g.fillna(g.mean(numeric_only=True)))

# Summary
print(f'Number of labeled points: {len(df)}')
df.head()

## 2. Model Training: Random Forest Classifier

We will now train a **Random Forest** classifier, a powerful and widely used supervised machine learning algorithm.

### How It Works:
- Builds an **ensemble of decision trees**
- Each tree is trained on a **random subset** of the data and features
- Final prediction is made by **aggregating** outputs from all trees
- Helps reduce **overfitting** and improves **generalization**


### Application in Landcover Classification

Each pixel (or label point) is assigned a landcover class based on:
- Spectral bands (Sentinel-2)
- Spectral indices (NDVI, NDBI)
- Radar backscatter (Sentinel-1)
- Elevation


### Model Evaluation Metrics

To assess model performance, we’ll use:

- **Accuracy**: Proportion of correctly classified samples
- **F1-score**: Harmonic mean of precision and recall (useful for imbalanced classes)
- **Cohen’s Kappa**: Measures agreement between predicted and true labels, accounting for chance

Together, these metrics provide a comprehensive view of how well the model distinguishes landcover types.


> ❓ **Question for you:**  
> Which metric do you think is most important for landcover classification in your context and why?
>
> 
> ##### Record your responses here: 

In [ ]:
# Count the frequency of each unique land cover type in the 'Land_Cover' column
# This helps us understand how many times each category appears in the dataset
counts = df["Land_Cover"].value_counts()

# Display the resulting counts
# This will show a Series with land cover types as the index and their respective counts as values
counts

In [ ]:
# Display the list of possible features for training a model
# We use slicing to exclude the last 4 columns from the list of all column names
print(f'Possible features for training:\n {df.columns.values[:-4]}')

In [ ]:
# Define the input features for the model.
# These are spectral bands and derived indices commonly used in remote sensing:
# - 'red', 'blue', 'green', 'nir08', 'swir16': spectral bands
# - 'ndvi', 'ndbi': vegetation and built-up indices
# - 'vh', 'vv': radar backscatter values
# - 'dem': elevation data
features = df[['red', 'blue', 'green', 'nir08', 'swir16', 'ndvi', 'ndbi', 'vh', 'vv', 'dem']]

# Define the target labels for classification.
# 'Class_id' represents the land cover class each sample belongs to.
labels = df['Class_id']

# Split the dataset into training and validation sets.
# - 80% of the data is used for training
# - 20% is used for validation
# - random_state ensures reproducibility of the split
X_train, X_val, y_train, y_val = train_test_split(
    features, labels, test_size=0.20, random_state=42)

In [ ]:
# Initialize the Random Forest model with 400 decision trees (n_estimators), a fixed random seed for reproducibility and 'balanced' class weights to handle class imbalance by adjusting weights inversely proportional to class frequencies
rf = RandomForestClassifier(
    n_estimators=400,
    random_state=42,
    class_weight='balanced')

# Train the model using the training data
rf.fit(X_train, y_train)

# Use the trained model to predict land cover classes for the validation set
y_pred = rf.predict(X_val)

# Calculate accuracy: proportion of correctly predicted samples
accuracy = accuracy_score(y_val, y_pred)

# Calculate weighted F1-score: harmonic mean of precision and recall, weighted by support of each class
f1 = f1_score(y_val, y_pred, average='weighted')

# Calculate Cohen's Kappa: measures agreement between predicted and true labels, adjusted for chance
kappa = cohen_kappa_score(y_val, y_pred)

# Generate a detailed classification report: includes precision, recall, F1-score, and support for each class
report = classification_report(y_val, y_pred)

# Display Results
print("Random Forest Classifier Performance")
print(f"Accuracy: {accuracy:.2%}")
print(f"Weighted F1-score: {f1:.2%}")
print(f"Cohen's Kappa: {kappa:.3f}")
print("\nClassification Report:\n", report)

In [ ]:
# Define the list of features used during training
train_features = ['red', 'blue', 'green', 'nir08', 'swir16', 'ndvi', 'ndbi', 'vh', 'vv', 'dem']

# Assume 'final_data' is an xarray.Dataset containing the raster layers for prediction
datasets = final_data

# Check if all required features are present in the dataset
missing_features = [f for f in train_features if f not in datasets.data_vars]
if missing_features:
    raise ValueError(f"Missing features in the dataset: {missing_features}")

# Stack all feature layers into a single NumPy array
# Each layer is squeezed to remove singleton dimensions
# The result is a 3D array with shape: (bands, height, width)
stacked = np.stack([np.squeeze(datasets[f].values) for f in train_features], axis=0)

# Extract dimensions
bands, ysize, xsize = stacked.shape

# Reshape the stacked array into a 2D array for prediction
# Shape becomes (num_pixels, num_features)
X_test = stacked.reshape(bands, -1).T

# Identify valid pixels (i.e., those without NaNs or infinite values)
valid_mask = np.all(np.isfinite(X_test), axis=1)

# Filter out invalid pixels for prediction
X_valid = X_test[valid_mask]

# Initialize an array to hold predictions for all pixels and fill with NaNs initially
pred_flat = np.full(X_test.shape[0], np.nan)

# Predict land cover class only for valid pixels
pred_flat[valid_mask] = rf.predict(X_valid)

# Convert the flat prediction array back to the original spatial dimensions
pred_raster = pred_flat.reshape(ysize, xsize)

In [ ]:
# Extract the importance scores for each feature
# These scores indicate how much each feature contributed to the model's decision-making
importances = rf.feature_importances_

# Get the names of the features used during training
feature_names = features.columns

# Create a DataFrame to organize and display feature importances
# Sort the features in descending order of importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Display the Feature Importance Table

print("\nFeature Importances:")
print(importance_df)

## Your Turn: Interpret Feature Importance and Validate Prediction

Now that you've generated the importance table, let's explore the features and and it's implications.

> ❓ **Question for you:**
> - What are the **top 3 most important features**? Why these features might be critical for distinguishing land cover classes
> - How do these features relate to the spectral or radar properties of different land cover types? Example: Why might `ndvi` be important for vegetation?
>
> 
> ##### Record your responses here: 


> ##### Record your responses here: 

## 3. Visualize Prediction

In [ ]:
# Convert the red, green, and blue bands to float32 for consistency
red = datasets["red"].astype("float32")
green = datasets["green"].astype("float32")
blue = datasets["blue"].astype("float32")

# Stack the three bands along the last axis to create an RGB image, the resulting shape will be (height, width, 3)
rgb = np.stack([red, green, blue], axis=-1)

# Normalize the RGB values using the 99th percentile - this enhances contrast by reducing the influence of extreme values
rgb_norm = np.clip(rgb / np.percentile(rgb, 99), 0, 1)

In [ ]:
# Dictionary mapping class IDs to descriptive names
class_labels = {
    1: "Forest",
    2: "Settlement",
    3: "Water",
    4: "Grassland/Shrubland"
}

# Define colors for each class in the visualization
colors = [
    "darkgreen",   # Forest
    "grey",        # Settlement
    "darkblue",    # Water
    "lightgreen"   # Grassland/Shrubland
]

# Create a colormap using the defined colors
cmap = ListedColormap(colors)

# Create a figure with two side-by-side subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Left subplot: RGB composite
axes[0].imshow(rgb_norm)
axes[0].set_title("RGB Composite", fontsize=14)
axes[0].axis("off")

# Right subplot: Predicted land cover map
im = axes[1].imshow(pred_raster, cmap=cmap)
axes[1].set_title("Predicted Land Cover", fontsize=14)
axes[1].axis("off")

# Add a legend for land cover classes
patches = [Patch(color=colors[i-1], label=class_labels[i]) for i in class_labels]
axes[1].legend(
    handles=patches,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    title="Land Cover",
    fontsize=10
)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

> **Question for you:**
> - Compare the **RGB composite** and the **predicted land cover map**. Do the predictions make sense visually? Identify areas where the model might have misclassified land cover.

> ##### Record your responses here: 

In [ ]:
# Save the figure to a file
fig.savefig("predicted_land_cover.png", dpi=300, bbox_inches="tight")
print("Figure exported as 'predicted_land_cover.png'")

⚠️ **TODO**
   - Remove the least important feature(s) from the dataset.
   - Retrain the Random Forest model.
   - Compare accuracy and F1-score before and after feature removal.
   - What changed? Why?

## 4. Conclusion 

This workshop demonstrated a complete workflow for integrating QField-collected data into a machine learning pipeline for landcover classification using Digital Earth Pacific tools. By leveraging multi-source Earth observation datasets such as **Sentinel-2 optical imagery, Sentinel-1 radar, and Copernicus elevation models**, we built a rich feature set to train a Random Forest classifier.
 
### Key Achievements
 
- **Comprehensive Data Preparation**: Defined an Area of Interest (AOI) over Auckland and processed spectral bands, vegetation and built-up indices (NDVI, NDBI), radar backscatter, and elevation data.
- **Model Training and Evaluation**: Trained a Random Forest model using interpolated features at labeled points and evaluated its performance using accuracy, F1-score, and Cohen’s Kappa. The model showed strong predictive capability across four landcover classes: *Forest*, *Settlement*, *Water*, and *Grassland/Shrubland*.
- **Feature Importance Analysis**: NDVI, elevation, and radar backscatter emerged as top contributors, underscoring the value of combining optical, radar, and topographic data.
 
### Balancing Features and Training Points
 
A critical consideration in this workflow is the trade-off between the number of features and the number of labeled training points. While adding more features (e.g., spectral bands, indices, radar layers) can improve model expressiveness and accuracy, it also increases the dimensionality of the data. If the number of labeled points is too small relative to the number of features, the model risks **overfitting**, learning noise instead of general patterns.
 
To mitigate this:
 
- Ensure enough labeled samples for each landcover class.
- Consider feature selection or dimensionality reduction to retain only the most informative features.
- Use balanced class weights and robust validation metrics to assess generalization.
 
This balance is especially important in Pacific Island contexts, where labelled data may be limited but environmental diversity is high.
 
### Final Thoughts
 
This notebook provides a practical and scalable approach to landcover classification using open-source tools and cloud-native geospatial data. Participants are encouraged to experiment further; refining AOIs, testing alternative models, and exploring temporal dynamics, to enhance environmental monitoring and decision-making across the region.
